In [ ]:
import data
import compression_entropy as ce
import random
from collections import Counter

In [ ]:
CONTEXT_WINDOW = 3
REMOVE_PUNCTUATION = True

In [ ]:
filename = '/home/pablo/Documents/GitHubRepos/paralleltext/bibles/corpus/eng-x-bible-world.txt'
bible = data.parse_pbc_bible(filename)

In [ ]:
tokenized = bible.tokenize(remove_punctuation=REMOVE_PUNCTUATION, lowercase=True)

In [ ]:
_, _, book_verses, _, _ = data.join_by_toc(tokenized.verse_tokens)

Within each book, shuffle the verses to avoid correlations between them.

In [ ]:
chosen_books=[40, 41, 42, 43, 44, 66]
truncate_books = False
selected_book_verses = ce.select_samples(book_verses, chosen_books, truncate_books)

In [ ]:
book_tokens = {book_id: random.sample(verses, k=len(verses)) \
               for book_id, verses in selected_book_verses.items()}

Now join all verses together so that we can chop them again.

In [ ]:
book_joined = {book_id: [el for lis in verses for el in lis] \
               for book_id, verses in book_tokens.items()}

Now chop each book into sequences of CONTEXT_WINDOW tokens.

In [ ]:
book_chopped = {book_id: [joined[i:i+CONTEXT_WINDOW] \
                          for i in range(0, len(joined) // CONTEXT_WINDOW, CONTEXT_WINDOW)] \
                for book_id, joined in book_joined.items()}

Now we can compute the counts for each of these sequences. Counter can't do this automatically because list is not hashable. So we have to convert these to strings. Because we know that the spaces are used for tokenization, we can safely insert a space between the tokens and we know it will not lead to any duplicates.

In [ ]:
book_joinchopped = {book_id: [' '.join(el) for el in chopped] \
                    for book_id, chopped in book_chopped.items()}

In [ ]:
book_counter = {book_id: Counter(joinchopped) \
                for book_id, joinchopped in book_joinchopped.items()}

In [ ]:
book_counter[43].most_common(10)

Thus we can see that a sequence length of 5 already leads to sequences that are too rare. We can repeat this analysis with different sequence lengths, and with or without punctuation.

**UPDATE**: removing punctuation did not cause a significant difference.

**UPDATE**: A context window of 3 makes sequences be more frequent, and might allow us to do some calculations.

However, this would be a computation of the entropy of generation of 3-token sequences, which is not the same as the entropy of generation of verses. So we are computing different things, and thus the estimator is not useful.